In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/공부/머신러닝/빅데이터 분석기사/Datasets/titanic/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

df = df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
df["Sex"] = LabelEncoder().fit_transform(df["Sex"])
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])
df["Embarked"] = df["Embarked"].fillna(method="ffill")
df["Age"] = MinMaxScaler().fit_transform(df[["Age"]])
df["Age"] = df["Age"].fillna(df["Age"].mean())
df["Fare"] = MinMaxScaler().fit_transform(df[["Fare"]])
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,0.271174,1,0,0.014151,2
1,1,1,0,0.472229,1,0,0.139136,0
2,1,3,0,0.321438,0,0,0.015469,2
3,1,1,0,0.434531,1,0,0.103644,2
4,0,3,1,0.434531,0,0,0.015713,2


In [ ]:
from sklearn.model_selection import train_test_split

label = df["Survived"]
data = df.drop("Survived", axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=0)

### 1-2

In [ ]:
import numpy as np

In [ ]:
X = np.arange(0, 10, 2)
Y = 1 / (1 + np.exp(-(3 * X + 4)))
w = np.random.randn(1)
b = np.zeros(1)
lr = 1

In [ ]:
for i in range(10000):
  dw = 0
  db = 0
  loss = 0
  for x in X:
    y = 1 / (1 + np.exp(-(3 * x + 4)))
    z = w * x + b
    a = 1 / (1 + np.exp(-z))
    L = - (y * np.log(a) + (1 - y) * np.log(1 - a))
    
    da = -y / a + (1 - y) / (1 - a)
    dz = a * (1 - a) * da
    dw += x * dz
    db += dz
    loss += L
  w -= lr * (dw / len(X))
  b -= lr * (db / len(X))
print(w, b)

[3.148139] [3.99933448]


In [ ]:
for i in range(10000):
  z = w * X + b
  a = 1 / (1 + np.exp(-z))
  L = np.sum(- (Y * np.log(a) + (1 - Y) * np.log(1 - a))) / X.shape[0]

  da = -Y / a + (1 - Y) / (1 - a)
  dz = a * (1 - a) * da
  dw = np.sum(X * dz) / X.shape[0]
  db = np.sum(dz) / X.shape[0]

  w -= lr * dw
  b -= lr * db
print(w, b)

[3.10724626] [3.99949858]


In [ ]:
X, Y = np.array(X_train), np.array(Y_train).reshape(1, -1)
W = np.random.randn(1, X.shape[1])
b = np.zeros((1, 1))
lr = 1

In [ ]:
for i in range(10000):
  Z = np.dot(W, X.T) + b
  A = 1 / (1 + np.exp(-Z))
  L = np.sum(- (Y * np.log(A) + (1 - Y) * np.log(1 - A))) / X.shape[0]

  dA = -Y / A + (1 - Y) / (1 - A)
  dZ = A * (1 - A) * dA
  dW = np.sum(X.T * dZ, axis=1, keepdims=True) / X.shape[0]
  db = np.sum(dZ, axis=1, keepdims=True) / X.shape[0]

  W = W - lr * dW.T
  b = b - lr * db
print(W, b, L)

[[-1.03721527 -2.72307139 -3.44251279 -0.38367333 -0.06934706  0.7855652
  -0.16867532]] [[5.24488291]] 0.4462955073840645


In [ ]:
X_test, Y_test = np.array(X_test), np.array(Y_test).reshape(1, -1)
Z = np.dot(W, X_test.T) + b
A = 1 / (1 + np.exp(-Z))
A = A > 0.5
np.sum(A == Y_test) / Y_test.shape[1]

0.7988826815642458

Logistic Regression 구현

In [ ]:
class LogisticRegression():
  def __init__(self, name='logistic_regression'):
    self.name = name
    self.W = None
    self.b = None

  def initialize_parameters(self, X):
    W = np.random.randn(1, X.shape[1])
    b = np.zeros((1, 1))
    return W, b

  def parameters(self):
    return self.W, self.b

  def sigmoid(self, x):
    return 1 / (1 + np.exp(-x))

  def binary_crossentropy(self, y, yhat):
    return np.sum(- (y * np.log(yhat) + (1 - y) * np.log(1 - yhat))) / y.shape[1]

  def forward_propagation(self, X, W, b):
    Z = np.dot(W, X.T) + b
    A = 1 / (1 + np.exp(-Z))
    return A

  def backward_propagation(self, X, Y, A, W, b):
    dA = -Y / A + (1 - Y) / (1 - A)
    dZ = A * (1 - A) * dA
    dW = np.sum(X.T * dZ, axis=1, keepdims=True) / X.shape[0]
    db = np.sum(dZ, axis=1, keepdims=True) / X.shape[0]
    return dW, db

  def fit(self, X, Y, epochs=10, learning_rate=0.1):
    if self.W == None:
      self.W, self.b = self.initialize_parameters(X)
      
    for _ in range(epochs):
      A = self.forward_propagation(X, self.W, self.b)
      loss = self.binary_crossentropy(Y, A)
      acc = np.sum((A > 0.5) == Y) / Y.shape[1]
      print("Train_loss : ", round(loss, 4), "Train_acc : ", round(acc, 4))

      dW, db = self.backward_propagation(X, Y, A, self.W, self.b)
      self.W = self.W - learning_rate * dW.T
      self.b = self.b - learning_rate * db

  def predict(self, X):
    return self.forward_propagation(X, self.W, self.b)

In [ ]:
X, Y = np.array(X_train), np.array(Y_train).reshape(1, -1)
model = LogisticRegression()
model.fit(X, Y, epochs=20, learning_rate=1)

Train_loss :  1.3942 Train_acc :  0.3876
Train_loss :  1.0098 Train_acc :  0.6166
Train_loss :  0.8101 Train_acc :  0.4551
Train_loss :  0.7641 Train_acc :  0.6419
Train_loss :  0.7107 Train_acc :  0.5632
Train_loss :  0.679 Train_acc :  0.6545
Train_loss :  0.6442 Train_acc :  0.7275
Train_loss :  0.6257 Train_acc :  0.6742
Train_loss :  0.6037 Train_acc :  0.7654
Train_loss :  0.5966 Train_acc :  0.6868
Train_loss :  0.5823 Train_acc :  0.7823
Train_loss :  0.5818 Train_acc :  0.7008
Train_loss :  0.5701 Train_acc :  0.7781
Train_loss :  0.5718 Train_acc :  0.7121
Train_loss :  0.5609 Train_acc :  0.7865
Train_loss :  0.5622 Train_acc :  0.7275
Train_loss :  0.5522 Train_acc :  0.7865
Train_loss :  0.5524 Train_acc :  0.7458
Train_loss :  0.5439 Train_acc :  0.7865
Train_loss :  0.5431 Train_acc :  0.7514


1-3

In [ ]:
import numpy as np

In [ ]:
X, Y = np.array(X_train), np.array(Y_train).reshape(1, -1)
layers = [X.shape[1], 4, Y.shape[0]]
W1 = np.random.randn(layers[1], layers[0])
b1 = np.zeros((layers[1], 1))
W2 = np.random.randn(layers[2], layers[1])
b2 = np.zeros((layers[2], 1))
learning_rate = 0.1
print(X.shape, Y.shape)
print(W1.shape, b1.shape)
print(W2.shape, b2.shape)

(712, 7) (1, 712)
(4, 7) (4, 1)
(1, 4) (1, 1)


In [ ]:
def tanh(x):
  return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [ ]:
for i in range(10000):
  Z1 = np.dot(W1, X.T) + b1
  A1 = tanh(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = sigmoid(Z2)
  loss = np.sum(- (Y * np.log(A2) + (1 - Y) * np.log(1 - A2))) / X.shape[0]

  dA2 = - Y / A2 + (1 - Y) / (1 - A2)
  dZ2 = A2 * (1 - A2) * dA2
  dW2 = np.dot(dZ2, A1.T) / X.shape[0]
  db2 = np.sum(dZ2, axis=1, keepdims=True) / X.shape[0]

  dA1 = np.dot(W2.T, dZ2)
  dZ1 = (1 - A1 * A1) * dA1
  dW1 = np.dot(dZ1, X) / X.shape[0]
  db1 = np.sum(dZ1, axis=1, keepdims=True) / X.shape[0]

  W2 = W2 - learning_rate * dW2
  b2 = b2 - learning_rate * db2
  W1 = W1 - learning_rate * dW1
  b1 = b1 - learning_rate * db1
print(loss)

1.474436001875799


1-4

In [ ]:
X, Y = np.array(X_train), np.array(Y_train).reshape(1, -1)
layers = [X.shape[1], 4, Y.shape[0]]
W1 = np.random.randn(layers[1], layers[0])
b1 = np.zeros((layers[1], 1))
W2 = np.random.randn(layers[2], layers[1])
b2 = np.zeros((layers[2], 1))
learning_rate = 0.1
print(X.shape, Y.shape)
print(W1.shape, b1.shape)
print(W2.shape, b2.shape)

In [ ]:
class NeuralNetwork():
  def __init__(self, layers, name='neural_network'):
    self.name = name
    self.layers = layers
    self.parameters = self.init_parameters(layers)
    self.grads = {}

  
  def binary_crossentropy(self, y, yhat):
    return np.sum(- (y * np.log(yhat) + (1 - y) * np.log(1 - yhat)))

  def init_parameters(self, layers):
    parameters = {}
    for l in range(1, len(layers)):
      parameters['W' + str(l)] = np.random.randn(layers[l-1], layers[l])
      parameters['b' + str(l)] = np.zeros((layers[l-1], 1))
    return parameters
  def forward_propagation(self, X, parameters):
    for i in range(1, len(self.layers)):

  
  def fit(self, X, Y, epochs=10, learning_rate=0.1):
    m = X.shape[0]
    for _ in range(epochs):
      self.grads = self.forward_propagation(X, self.parameters)
      loss = 

In [ ]:
class Dense():
  def __init__(self, units=4, activation='relu'):
    self.W
    self.b
    self.activation

  def forward_propagation(self, X):
    Z = np.dot(self.W, X.T) + self.b
    A = self.activation(Z, 'forward')
    return A

  def backward_propagation(self, X, Y, A):
    dA = -Y / A + (1 - Y) / (1 - A)
    dZ = self.activation(A, 'backward') * dA
    dW = np.sum(X.T * dZ, axis=1, keepdims=True) / X.shape[0]
    db = np.sum(dZ, axis=1, keepdims=True) / X.shape[0]
    return dW, db

In [ ]:
class activation_fuction():
  def sigmoid(self, x):
    return 1 / (1 + np.exp(-x))
  def tanh(self, x):
    return (np.exp(x) - np.exp(-x) / np.exp(x) + np.exp(-x))
  def relu(self, x):
    return x if x > 0 else 0